### Installation

In [1]:
import torch
import os
print("PyTorch has version {}".format(torch.__version__))

# Install torch geometric
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
!pip install torch-geometric
!pip install ogb

PyTorch has version 1.13.1+cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 12.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 16.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 28.1 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=4628e68cedf9c70454607507570ccd2f3e3c189f2961088714926

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7047 sha256=5eba3abdd58f0b7138126f5735dd584ef8dea9de042c335525b25c1b3dcca44b
  Stored in directory: /root/.cache/pip/wheels/04/bb/0d/2d02ec45f29c48d6192476bfb59c5a0e64b605e7212374dd15
Successfully built littleutils


### Load datasets

In [2]:
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

from ogb.linkproppred import PygLinkPropPredDataset

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
def load_node_dataset(dataset_name):
    # Load the dataset and transform it to sparse tensor
    dataset = PygNodePropPredDataset(name=dataset_name,
                                    transform=T.ToSparseTensor())
    print('The {} dataset has {} graph'.format(dataset_name, len(dataset)))

    # Split
    split_idx = dataset.get_idx_split()

    # Extract the graph
    data = dataset[0]
    data = data.to(device)

    print('Task type: {}'.format(dataset.task_type))

    return dataset, split_idx, data 


In [5]:
# load edge dataset 
d_name = 'ogbl-collab'
dataset = PygLinkPropPredDataset(name = d_name) 

split_edge = dataset.get_edge_split()
train_edge, valid_edge, test_edge = split_edge["train"], split_edge["valid"], split_edge["test"]
graph = dataset[0] # pyg graph object containing only training edges



Downloaded 0.11 GB: 100%|██████████| 117/117 [00:02<00:00, 51.69it/s]


Extracting dataset/collab.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00,  6.44it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 148.77it/s]

Saving...



Done!


In [6]:
dataset_arxiv, split_arxiv, data_arxiv = load_node_dataset(dataset_name='ogbn-arxiv')

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:02<00:00, 39.71it/s]


Extracting dataset/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 5269.23it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 3653.57it/s]

Saving...



Done!


The ogbn-arxiv dataset has 1 graph
Task type: multiclass classification


### Baseline GNNs

In [7]:
import torch
import torch_scatter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils
from torch_geometric.nn import Sequential, GCNConv

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType,
                                    OptTensor)

from torch.nn import BatchNorm1d, Dropout, Linear, Parameter, ReLU
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax, degree

from torch_geometric.nn import GCNConv,GATConv,GATv2Conv, GINConv
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import copy
import pickle

In [12]:
def train(model, dataset, args):
    
    print()
    test_loader = loader = DataLoader(dataset, batch_size=args['batch_size'], shuffle=False)

    opt = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])
    # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.opt_decay_step, gamma=args.opt_decay_rate)
    loss_fn = F.nll_loss

    # train
    losses = []
    test_accs = []
    best_acc = 0
    best_model = None
    for epoch in range(args['epochs']):
        total_loss = 0
        model.train()
        for batch in loader:
          print(batch.edge_index)
          opt.zero_grad()
          pred = model(batch.x, batch.edge_index)
          label = batch.y
          pred = pred[batch.train_mask]
          label = label[batch.train_mask]
          loss = loss_fn(pred, label)
          loss.backward()
          opt.step()
          total_loss += loss.item() * batch.num_graphs
        total_loss /= len(loader.dataset)
        losses.append(total_loss)

        test_acc = test(test_loader, model)
        test_accs.append(test_acc)
        if test_acc > best_acc:
          best_acc = test_acc
          best_model = copy.deepcopy(model)
    
    return test_accs, losses, best_model, best_acc, test_loader

def test(loader, test_model, is_validation=False, save_model_preds=False, model_type=None):
    test_model.eval()

    correct = 0
    for data in loader:
        with torch.no_grad():
            # max(dim=1) returns values, indices tuple; only need indices
            pred = test_model(data).max(dim=1)[1]
            label = data.y

        mask = data.val_mask if is_validation else data.test_mask
        # node classification: only evaluate on nodes in test set
        pred = pred[mask]
        label = label[mask]

        if save_model_preds:
          print ("Saving Model Predictions for Model Type", model_type)

          data = {}
          data['pred'] = pred.view(-1).cpu().detach().numpy()
          data['label'] = label.view(-1).cpu().detach().numpy()
            
        correct += pred.eq(label).sum().item()

    total = 0
    for data in loader.dataset:
        total += torch.sum(data.val_mask if is_validation else data.test_mask).item()

    return correct / total

In [13]:
def evaluate_model(model, args):
  model.reset_parameters()

  test_accs, losses, best_model, best_acc, test_loader = train(model, dataset_arxiv, args) 

  print("Maximum test set accuracy: {0}".format(max(test_accs)))
  print("Minimum loss: {0}".format(min(losses)))

  # Run test for our best model to save the predictions!
  test(test_loader, best_model, is_validation=False, save_model_preds=True, model_type=model)
  print()

  plt.title(f"{model.name} obgn-arxiv node classification")
  plt.plot(losses, label="training loss" + " - " + args.model_type)
  plt.plot(test_accs, label="test accuracy" + " - " + args.model_type)
  plt.legend()
  plt.show()


In [14]:
class GCN(torch.nn.Module):
  # 3-layer GCN
  def __init__(self, input_dim, hidden_dim, output_dim, dropout):
    super(GCN, self).__init__()
    self.name = "GCN"
    self.model = Sequential('x, edge_index', [
        (GCNConv(input_dim, hidden_dim), 'x, edge_index -> x'),
        BatchNorm1d(hidden_dim),
        ReLU(),
        Dropout(p=dropout),
        (GCNConv(hidden_dim, hidden_dim), 'x, edge_index -> x'),
        BatchNorm1d(hidden_dim),
        ReLU(),
        Dropout(p=dropout),
        (GCNConv(hidden_dim, hidden_dim), 'x, edge_index -> x'),
        BatchNorm1d(hidden_dim),
        ReLU(),
        Dropout(p=dropout),
        (GCNConv(hidden_dim, output_dim), 'x, edge_index -> x'),
        nn.LogSoftmax(1),
    ])
  
  def reset_parameters(self):
    for layer in self.model.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
  
  def forward(self, x, adj_t):
    return self.model(x, adj_t)

In [15]:
gcn_args = {
  'device': device,
  'hidden_dim': 256,
  'batch_size': 32,
  'dropout': 0.5,
  'weight_decay': 5e-3,
  'lr': 0.01,
  'epochs': 100,
}
gcn = GCN(dataset_arxiv.num_features, gcn_args['hidden_dim'], dataset_arxiv.num_classes, dropout=gcn_args['dropout']).to(device)

evaluate_model(gcn, gcn_args)


None


ValueError: ignored

In [ ]:
torch.cuda.empty_cache()

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads, concat, dropout):
        super(GAT, self).__init__()
        self.model = Sequential('x, edge_index', [
            (GATConv(input_dim, hidden_dim, heads=num_heads, concat=concat), 'x, edge_index -> x'),
            BatchNorm1d(hidden_dim),
            ReLU(),
            Dropout(p=dropout),
            (GATConv(hidden_dim, hidden_dim, heads=num_heads, concat=concat), 'x, edge_index -> x'),
            BatchNorm1d(hidden_dim),
            ReLU(),
            Dropout(p=dropout),
            (GATConv(hidden_dim, hidden_dim, heads=num_heads, concat=concat), 'x, edge_index -> x'),
            BatchNorm1d(hidden_dim),
            ReLU(),
            Dropout(p=dropout),
            (GATConv(hidden_dim, output_dim, heads=num_heads, concat=concat), 'x, edge_index -> x'),
            nn.LogSoftmax(1),
        ])
    
    def reset_parameters(self):
        for layer in self.model.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()
    
    def forward(self, x, adj_t):
        return self.model(x, adj_t)

In [ ]:
gat_args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 256,
    'num_heads': 8,
    'concat': True,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
}
gat = GAT(dataset_arxiv.num_features, gat_args['hidden_dim'], dataset_arxiv.num_classes, gat_args['num_heads'], gat_args['concat'], dropout=gat_args['dropout']).to(device)
evaluate_model(gat, gat_args)

In [ ]:
class GIN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout):
        super(GIN, self).__init__()
        self.name = "GIN"
        self.model = Sequential('x, edge_index', [
            (GINConv(Sequential('x, edge_index', [
                (GCNConv(input_dim, hidden_dim), 'x, edge_index -> x'),
                BatchNorm1d(hidden_dim),
                ReLU(),
                Dropout(p=dropout),
            ])), 'x, edge_index -> x'),
            BatchNorm1d(hidden_dim),
            ReLU(),
            Dropout(p=dropout),
            (GINConv(Sequential('x, edge_index', [
                (GCNConv(hidden_dim, hidden_dim), 'x, edge_index -> x'),
                BatchNorm1d(hidden_dim),
                ReLU(),
                Dropout(p=dropout),
            ])), 'x, edge_index -> x'),
            BatchNorm1d(hidden_dim),
            ReLU(),
            Dropout(p=dropout),
            (GINConv(Sequential('x, edge_index', [
                (GCNConv(hidden_dim, hidden_dim), 'x, edge_index -> x'),
                BatchNorm1d(hidden_dim),
                ReLU(),
                Dropout(p=dropout),
            ])), 'x, edge_index -> x'),
            BatchNorm1d(hidden_dim),
            ReLU(),
            Dropout(p=dropout),
            (GCNConv(hidden_dim, output_dim), 'x, edge_index -> x'),
            nn.LogSoftmax(1),
        ])
    
    def reset_parameters(self):
        for layer in self.model.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()
    
    def forward(self, x, adj_t):
        return self.model(x, adj_t)

In [ ]:
gin_args = {
    'device': device,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
}
gin = GIN(dataset_arxiv.num_features, gin_args['hidden_dim'], dataset_arxiv.num_classes, dropout=gin_args['dropout']).to(device)

evaluate_model(gin, gin_args)

### Graph CL Augmentations
